In [ ]:
import pandas as pd

df = pd.read_csv('data/history.csv')
#print(df.head())
#print(df.columns)

#Specifying columns to check for zeros
cols_to_check = ["sunshine_duration (s)", "daylight_duration (s)", "shortwave_radiation_sum (MJ/m²)"]

#Checking for zeros in the specified columns
rows_with_zero = df[(df[cols_to_check] == 0).any(axis=1)]

print(rows_with_zero)

             time  temperature_2m_mean (°C)  temperature_2m_max (°C)  \
143    1994-05-24                      26.0                     28.9   
178    1994-06-28                      27.0                     28.2   
494    1995-05-10                      27.1                     30.1   
497    1995-05-13                      26.2                     27.3   
500    1995-05-16                      26.1                     26.4   
...           ...                       ...                      ...   
11104  2024-05-27                      27.0                     27.8   
11170  2024-08-01                      26.9                     27.9   
11215  2024-09-15                      25.8                     26.9   
11255  2024-10-25                      25.1                     25.8   
11517  2025-07-14                      26.3                     27.8   

       temperature_2m_min (°C)  sunshine_duration (s)  daylight_duration (s)  \
143                       24.4                    0.0  

In [11]:
import pandas as pd

# If you uncomment the header line in history.csv first, use this simpler version:
df = pd.read_csv('data/history.csv')

# Filter rows where sunshine_duration is 0.0
zero_sunshine_rows = df[df['sunshine_duration (s)'] == 0.0]

# Save to a new CSV file
zero_sunshine_rows.to_csv('data/zero_sunshine_days.csv', index=False)

print(f"Found {len(zero_sunshine_rows)} days with zero sunshine duration")
print(f"Results saved to 'data/zero_sunshine_days.csv'")
print("\nPreview:")
print(zero_sunshine_rows[['time', 'sunshine_duration (s)', 'temperature_2m_mean (°C)']].head())

Found 172 days with zero sunshine duration
Results saved to 'data/zero_sunshine_days.csv'

Preview:
           time  sunshine_duration (s)  temperature_2m_mean (°C)
143  1994-05-24                    0.0                      26.0
178  1994-06-28                    0.0                      27.0
494  1995-05-10                    0.0                      27.1
497  1995-05-13                    0.0                      26.2
500  1995-05-16                    0.0                      26.1


In [12]:
import pandas as pd
import requests
from datetime import datetime
import time
from tqdm import tqdm  # For progress bar

# Read the zero sunshine days
zero_sunshine_df = pd.read_csv('data/zero_sunshine_days.csv')

# Read the main history file
history_df = pd.read_csv('data/history.csv', comment='#', header=None, skiprows=1)
history_df.columns = ['time', 'temperature_2m_mean (°C)', 'temperature_2m_max (°C)', 
                      'temperature_2m_min (°C)', 'sunshine_duration (s)', 
                      'daylight_duration (s)', 'shortwave_radiation_sum (MJ/m²)']

# Ensure 'time' column is datetime
zero_sunshine_df['time'] = pd.to_datetime(zero_sunshine_df['time'])
history_df['time'] = pd.to_datetime(history_df['time'])

# Base API URL
base_url = "https://archive-api.open-meteo.com/v1/archive"

# Location coordinates
params = {
    'latitude': 22.5626,
    'longitude': 88.363,
    'daily': 'temperature_2m_mean,temperature_2m_max,temperature_2m_min,sunshine_duration,daylight_duration,shortwave_radiation_sum',
    'timezone': 'GMT'
}

# Function to fetch data with retry logic and exponential backoff
def fetch_api_data(start_date, end_date, max_retries=3):
    params_copy = params.copy()
    params_copy['start_date'] = start_date
    params_copy['end_date'] = end_date
    
    for attempt in range(max_retries):
        try:
            response = requests.get(base_url, params=params_copy, timeout=30)
            
            # Handle rate limiting
            if response.status_code == 429:
                wait_time = 2 ** attempt  # Exponential backoff
                print(f"  ⚠️  Rate limited. Waiting {wait_time}s before retry...")
                time.sleep(wait_time)
                continue
            
            response.raise_for_status()
            data = response.json()
            
            # Convert to DataFrame
            df = pd.DataFrame({
                'time': pd.to_datetime(data['daily']['time']),
                'temperature_2m_mean (°C)': data['daily']['temperature_2m_mean'],
                'temperature_2m_max (°C)': data['daily']['temperature_2m_max'],
                'temperature_2m_min (°C)': data['daily']['temperature_2m_min'],
                'sunshine_duration (s)': data['daily']['sunshine_duration'],
                'daylight_duration (s)': data['daily']['daylight_duration'],
                'shortwave_radiation_sum (MJ/m²)': data['daily']['shortwave_radiation_sum']
            })
            
            return df
            
        except requests.exceptions.Timeout:
            print(f"  ⚠️  Timeout on attempt {attempt + 1}/{max_retries}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)
        except Exception as e:
            print(f"  ❌ Error: {e}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)
    
    return None

# Optimize: Group by YEAR instead of month (fewer API calls)
# Open-Meteo allows up to 1 year of data per request
zero_sunshine_df['year'] = zero_sunshine_df['time'].dt.year
years = sorted(zero_sunshine_df['year'].unique())

updated_count = 0
corrections = []

print(f"📊 Processing {len(zero_sunshine_df)} rows across {len(years)} years")
print(f"🌐 Making approximately {len(years)} API calls...\n")

for year in tqdm(years, desc="Fetching data by year"):
    # Get all dates for this year
    year_data = zero_sunshine_df[zero_sunshine_df['year'] == year]
    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"
    
    print(f"\n📅 Fetching year {year} ({len(year_data)} days to check)...")
    
    api_data = fetch_api_data(start_date, end_date)
    
    if api_data is not None:
        # Update history_df with API data for matching dates
        for date in year_data['time']:
            api_row = api_data[api_data['time'] == date]
            
            if not api_row.empty:
                api_row = api_row.iloc[0]
                hist_mask = history_df['time'] == date
                
                if hist_mask.any():
                    old_sunshine = history_df.loc[hist_mask, 'sunshine_duration (s)'].values[0]
                    new_sunshine = api_row['sunshine_duration (s)']
                    
                    # Only update if old value was 0 and new value is different
                    if old_sunshine == 0.0 and new_sunshine is not None:
                        # Update all columns
                        history_df.loc[hist_mask, 'sunshine_duration (s)'] = new_sunshine
                        history_df.loc[hist_mask, 'temperature_2m_mean (°C)'] = api_row['temperature_2m_mean (°C)']
                        history_df.loc[hist_mask, 'temperature_2m_max (°C)'] = api_row['temperature_2m_max (°C)']
                        history_df.loc[hist_mask, 'temperature_2m_min (°C)'] = api_row['temperature_2m_min (°C)']
                        history_df.loc[hist_mask, 'daylight_duration (s)'] = api_row['daylight_duration (s)']
                        history_df.loc[hist_mask, 'shortwave_radiation_sum (MJ/m²)'] = api_row['shortwave_radiation_sum (MJ/m²)']
                        
                        corrections.append({
                            'date': date.strftime('%Y-%m-%d'),
                            'old_sunshine': old_sunshine,
                            'new_sunshine': new_sunshine,
                            'difference': new_sunshine - old_sunshine
                        })
                        updated_count += 1
                        print(f"  ✅ {date.strftime('%Y-%m-%d')}: {old_sunshine} → {new_sunshine}")
    
    # Rate limit: Wait 1 second between requests (Open-Meteo free tier allows ~10,000 calls/day)
    if year != years[-1]:  # Don't wait after the last request
        time.sleep(1)

# Save results
print(f"\n💾 Saving results...")
history_df.to_csv('data/history_corrected.csv', index=False)

if corrections:
    corrections_df = pd.DataFrame(corrections)
    corrections_df.to_csv('data/corrections_log.csv', index=False)
    
    print(f"\n✅ SUCCESS!")
    print(f"📝 Updated {updated_count} rows")
    print(f"📂 Corrected history: data/history_corrected.csv")
    print(f"📋 Corrections log: data/corrections_log.csv")
    print(f"\n📊 Statistics:")
    print(f"   - Total sunshine added: {corrections_df['difference'].sum():.2f} seconds")
    print(f"   - Average correction: {corrections_df['difference'].mean():.2f} seconds")
else:
    print(f"\n⚠️  No corrections needed - all zero values confirmed!")

📊 Processing 172 rows across 32 years
🌐 Making approximately 32 API calls...



Fetching data by year:   0%|          | 0/32 [00:00<?, ?it/s]


📅 Fetching year 1994 (2 days to check)...
  ✅ 1994-05-24: 0.0 → 0.0
  ✅ 1994-06-28: 0.0 → 0.0


Fetching data by year:   3%|▎         | 1/32 [00:03<01:38,  3.18s/it]


📅 Fetching year 1995 (7 days to check)...
  ✅ 1995-05-10: 0.0 → 0.0
  ✅ 1995-05-13: 0.0 → 0.0
  ✅ 1995-05-16: 0.0 → 0.0
  ✅ 1995-07-19: 0.0 → 0.0
  ✅ 1995-10-30: 0.0 → 0.0
  ✅ 1995-11-01: 0.0 → 0.0
  ✅ 1995-11-09: 0.0 → 0.0


...

Fetching data by year:  97%|█████████▋| 31/32 [01:19<00:02,  2.47s/it]


📅 Fetching year 2025 (1 days to check)...
  ❌ Error: 400 Client Error: Bad Request for url: https://archive-api.open-meteo.com/v1/archive?latitude=22.5626&longitude=88.363&daily=temperature_2m_mean%2Ctemperature_2m_max%2Ctemperature_2m_min%2Csunshine_duration%2Cdaylight_duration%2Cshortwave_radiation_sum&timezone=GMT&start_date=2025-01-01&end_date=2025-12-31
  ❌ Error: 400 Client Error: Bad Request for url: https://archive-api.open-meteo.com/v1/archive?latitude=22.5626&longitude=88.363&daily=temperature_2m_mean%2Ctemperature_2m_max%2Ctemperature_2m_min%2Csunshine_duration%2Cdaylight_duration%2Cshortwave_radiation_sum&timezone=GMT&start_date=2025-01-01&end_date=2025-12-31


Fetching data by year: 100%|██████████| 32/32 [01:27<00:00,  2.72s/it]

  ❌ Error: 400 Client Error: Bad Request for url: https://archive-api.open-meteo.com/v1/archive?latitude=22.5626&longitude=88.363&daily=temperature_2m_mean%2Ctemperature_2m_max%2Ctemperature_2m_min%2Csunshine_duration%2Cdaylight_duration%2Cshortwave_radiation_sum&timezone=GMT&start_date=2025-01-01&end_date=2025-12-31

💾 Saving results...



✅ SUCCESS!
📝 Updated 171 rows
📂 Corrected history: data/history_corrected.csv
📋 Corrections log: data/corrections_log.csv

📊 Statistics:
   - Total sunshine added: 0.00 seconds
   - Average correction: 0.00 seconds


In [13]:
import pandas as pd
import requests
from datetime import datetime
import time

# Read the zero sunshine days
zero_sunshine_df = pd.read_csv('data/zero_sunshine_days.csv')

# Read the main history file
history_df = pd.read_csv('data/history.csv', comment='#', header=None, skiprows=1)
history_df.columns = ['time', 'temperature_2m_mean (°C)', 'temperature_2m_max (°C)', 
                      'temperature_2m_min (°C)', 'sunshine_duration (s)', 
                      'daylight_duration (s)', 'shortwave_radiation_sum (MJ/m²)']

# Ensure 'time' column is datetime
zero_sunshine_df['time'] = pd.to_datetime(zero_sunshine_df['time'])

# Group dates into ranges for API efficiency (API allows date ranges)
dates_to_check = zero_sunshine_df['time'].dt.date.tolist()

# Base API URL
base_url = "https://archive-api.open-meteo.com/v1/archive"

# Location coordinates
params = {
    'latitude': 22.5626,
    'longitude': 88.363,
    'daily': 'temperature_2m_mean,temperature_2m_max,temperature_2m_min,sunshine_duration,daylight_duration,shortwave_radiation_sum',
    'timezone': 'GMT'
}

# Function to fetch data for a date range
def fetch_api_data(start_date, end_date):
    params['start_date'] = start_date
    params['end_date'] = end_date
    
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        
        # Convert to DataFrame
        df = pd.DataFrame({
            'time': data['daily']['time'],
            'temperature_2m_mean (°C)': data['daily']['temperature_2m_mean'],
            'temperature_2m_max (°C)': data['daily']['temperature_2m_max'],
            'temperature_2m_min (°C)': data['daily']['temperature_2m_min'],
            'sunshine_duration (s)': data['daily']['sunshine_duration'],
            'daylight_duration (s)': data['daily']['daylight_duration'],
            'shortwave_radiation_sum (MJ/m²)': data['daily']['shortwave_radiation_sum']
        })
        
        return df
    except Exception as e:
        print(f"Error fetching data for {start_date} to {end_date}: {e}")
        return None

# Process dates in monthly chunks to minimize API calls
updated_count = 0
corrections = []

# Group by year-month
zero_sunshine_df['year_month'] = zero_sunshine_df['time'].dt.to_period('M')
grouped = zero_sunshine_df.groupby('year_month')

for period, group in grouped:
    start_date = group['time'].min().strftime('%Y-%m-%d')
    end_date = group['time'].max().strftime('%Y-%m-%d')
    
    print(f"Fetching data for {start_date} to {end_date}...")
    
    api_data = fetch_api_data(start_date, end_date)
    
    if api_data is not None:
        api_data['time'] = pd.to_datetime(api_data['time'])
        
        # Update history_df with API data for these dates
        for idx, row in api_data.iterrows():
            date = row['time']
            
            # Find matching row in history_df
            mask = pd.to_datetime(history_df['time']) == date
            
            if mask.any():
                old_sunshine = history_df.loc[mask, 'sunshine_duration (s)'].values[0]
                new_sunshine = row['sunshine_duration (s)']
                
                if old_sunshine == 0.0 and new_sunshine != 0.0:
                    # Update the values
                    history_df.loc[mask, 'sunshine_duration (s)'] = new_sunshine
                    history_df.loc[mask, 'temperature_2m_mean (°C)'] = row['temperature_2m_mean (°C)']
                    history_df.loc[mask, 'temperature_2m_max (°C)'] = row['temperature_2m_max (°C)']
                    history_df.loc[mask, 'temperature_2m_min (°C)'] = row['temperature_2m_min (°C)']
                    history_df.loc[mask, 'daylight_duration (s)'] = row['daylight_duration (s)']
                    history_df.loc[mask, 'shortwave_radiation_sum (MJ/m²)'] = row['shortwave_radiation_sum (MJ/m²)']
                    
                    corrections.append({
                        'date': date.strftime('%Y-%m-%d'),
                        'old_sunshine': old_sunshine,
                        'new_sunshine': new_sunshine
                    })
                    updated_count += 1
                    print(f"  Updated {date.strftime('%Y-%m-%d')}: {old_sunshine} → {new_sunshine}")
    
    # Be respectful to the API - add a small delay
    time.sleep(0.5)

# Save corrected history
history_df.to_csv('data/history_corrected.csv', index=False)

# Save correction log
if corrections:
    corrections_df = pd.DataFrame(corrections)
    corrections_df.to_csv('data/corrections_log.csv', index=False)

print(f"\n✅ Correction complete!")
print(f"Updated {updated_count} rows")
print(f"Corrected history saved to: data/history_corrected.csv")
print(f"Corrections log saved to: data/corrections_log.csv")

Fetching data for 1994-05-24 to 1994-05-24...
Fetching data for 1994-06-28 to 1994-06-28...
Fetching data for 1995-05-10 to 1995-05-16...
Fetching data for 1995-07-19 to 1995-07-19...
Fetching data for 1995-10-30 to 1995-10-30...
Fetching data for 1995-11-01 to 1995-11-09...
Fetching data for 1996-06-19 to 1996-06-30...
Fetching data for 1996-08-07 to 1996-08-21...
Fetching data for 1996-10-27 to 1996-10-28...
Fetching data for 1997-07-19 to 1997-07-20...
Fetching data for 1997-09-06 to 1997-09-26...
Fetching data for 1998-08-13 to 1998-08-13...
Fetching data for 1998-09-08 to 1998-09-12...
Fetching data for 1998-10-20 to 1998-10-20...
Fetching data for 1998-11-22 to 1998-11-22...
Fetching data for 1999-08-15 to 1999-08-15...
Fetching data for 1999-10-17 to 1999-10-17...
Fetching data for 2000-08-31 to 2000-08-31...
Fetching data for 2000-09-18 to 2000-09-18...
Fetching data for 2001-06-02 to 2001-06-02...
Fetching data for 2001-07-02 to 2001-07-02...
Fetching data for 2001-11-11 to 20

In [14]:
import pandas as pd
import requests

# Read the zero sunshine days
zero_sunshine_df = pd.read_csv('data/zero_sunshine_days.csv')

# Get the first date with zero sunshine
test_date = zero_sunshine_df['time'].iloc[0]
print(f"🔍 Testing with date: {test_date}")

# Read your current history data for comparison
history_df = pd.read_csv('data/history.csv', comment='#', header=None, skiprows=1)
history_df.columns = ['time', 'temperature_2m_mean (°C)', 'temperature_2m_max (°C)', 
                      'temperature_2m_min (°C)', 'sunshine_duration (s)', 
                      'daylight_duration (s)', 'shortwave_radiation_sum (MJ/m²)']

# Get the existing values for this date
existing_row = history_df[history_df['time'] == test_date]

print("\n📊 YOUR CURRENT DATA:")
print(existing_row.to_string(index=False))

# Fetch from API
api_url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    'latitude': 22.5626,
    'longitude': 88.363,
    'start_date': test_date,
    'end_date': test_date,
    'daily': 'temperature_2m_mean,temperature_2m_max,temperature_2m_min,sunshine_duration,daylight_duration,shortwave_radiation_sum',
    'timezone': 'GMT'
}

try:
    response = requests.get(api_url, params=params)
    response.raise_for_status()
    data = response.json()
    
    print("\n🌐 API DATA:")
    print(f"Date: {data['daily']['time'][0]}")
    print(f"Temperature Mean: {data['daily']['temperature_2m_mean'][0]} °C")
    print(f"Temperature Max: {data['daily']['temperature_2m_max'][0]} °C")
    print(f"Temperature Min: {data['daily']['temperature_2m_min'][0]} °C")
    print(f"Sunshine Duration: {data['daily']['sunshine_duration'][0]} s")
    print(f"Daylight Duration: {data['daily']['daylight_duration'][0]} s")
    print(f"Shortwave Radiation: {data['daily']['shortwave_radiation_sum'][0]} MJ/m²")
    
    # Compare
    print("\n🔄 COMPARISON:")
    your_sunshine = existing_row['sunshine_duration (s)'].values[0] if not existing_row.empty else None
    api_sunshine = data['daily']['sunshine_duration'][0]
    
    print(f"Your sunshine_duration: {your_sunshine} s")
    print(f"API sunshine_duration: {api_sunshine} s")
    
    if your_sunshine == 0.0 and api_sunshine != 0.0:
        print(f"✅ CHANGE DETECTED! API has non-zero value: {api_sunshine} s")
    elif your_sunshine == api_sunshine:
        print(f"⚠️  NO CHANGE - Both have same value: {your_sunshine} s")
    else:
        print(f"📝 Different values found")
        
except Exception as e:
    print(f"❌ Error fetching API data: {e}")

🔍 Testing with date: 1994-05-24

📊 YOUR CURRENT DATA:
      time  temperature_2m_mean (°C)  temperature_2m_max (°C)  temperature_2m_min (°C)  sunshine_duration (s)  daylight_duration (s)  shortwave_radiation_sum (MJ/m²)
1994-05-24                      26.0                     28.9                     24.4                    0.0               48016.72                             5.98

🌐 API DATA:
Date: 1994-05-24
Temperature Mean: 26.0 °C
Temperature Max: 28.9 °C
Temperature Min: 24.4 °C
Sunshine Duration: 0.0 s
Daylight Duration: 48016.72 s
Shortwave Radiation: 5.98 MJ/m²

🔄 COMPARISON:
Your sunshine_duration: 0.0 s
API sunshine_duration: 0.0 s
⚠️  NO CHANGE - Both have same value: 0.0 s


In [16]:
# Rename columns (modify keys if needed)
df = df.rename(columns={
    "time": "date",
    "temperature_2m_mean": "temp_mean",
    "temperature_2m_max": "temp_max",
    "temperature_2m_min": "temp_min"
})

print(df.head())


         date  temperature_2m_mean (°C)  temperature_2m_max (°C)  \
0  1994-01-01                      20.0                     27.2   
1  1994-01-02                      20.0                     27.1   
2  1994-01-03                      20.7                     28.2   
3  1994-01-04                      21.0                     27.7   
4  1994-01-05                      21.6                     28.0   

   temperature_2m_min (°C)  sunshine_duration (s)  daylight_duration (s)  \
0                     13.9               35388.50               38813.64   
1                     13.4               35337.72               38834.01   
2                     14.2               35348.36               38856.03   
3                     14.4               35392.11               38879.66   
4                     14.7               35311.84               38904.88   

   shortwave_radiation_sum (MJ/m²)  
0                            16.70  
1                            16.64  
2                      